In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pickle
import pandas as pd

In [2]:
train_set=pd.read_pickle('dataset_next_state/integrators_next_state_train.pickle')
test_set=pd.read_pickle('dataset_next_state/integrators_next_state_test.pickle')

In [4]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.fc = nn.Linear(output_size, output_size)

    def forward(self, input, hidden):
        print(input)
        print(hidden)
        combined = torch.cat((input, hidden))
        hidden = self.i2h(combined)
        output = self.h2o(hidden)
        output = self.fc(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(self.hidden_size)

In [8]:
def train(step,hidden,result,nn,learning_rate,loss_function):
    nn.zero_grad()
    output, hidden = nn(step, hidden)
    loss = loss_function(output, result)
    loss.backward()
    for p in nn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)
    return hidden,loss.item()

In [6]:
n_iters = 1000
print_every = 100
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn
loss_function = nn.MSELoss()
n_hidden = 10
dimension=train_set[0][0].size(0)
rnn = RNN(dimension,n_hidden,dimension)

In [9]:
losses=[]
count=0
for iter in range(1, n_iters + 1):
    for trace in train_set:
        hidden=rnn.initHidden()
        for i in range(len(trace)-1):           
            hidden,loss = train(trace[i],hidden,trace[i+1],rnn,learning_rate,loss_function)
            losses.append(loss)
        if count % print_every == 0:
            print(loss)
        count+=1

tensor([ 18.1149, 142.1318,   4.5874,   0.0000,   1.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   1.5149])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
nan
tensor([1.6833e+01, 1.4089e+02, 2.8766e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 3.4712e+00])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([ 16.8325, 140.8878,   3.5000,   0.0000,   1.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.2325])
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       grad_fn=<ViewBackward0>)


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.